# Ludwig with Bert and GloVe on V5 data

- V5_Data: Target: Google Trend, Row: Hourly articles, Time: 2015 - 2020, 12 hour shift
- Utlizes Ludwig package, which is essentially a Tensorflow wrapper with out-of-box architecture built by uber
- Did not perform well but I think this package has a lot of potential because it makes it relatively easy to create ensemble models
- Results in 'results' folder
- Experimenting with Bert transfomer, GloVe embedding, and stacked GloVe with sentiment scores

# Installs

# Installs and Imports

In [1]:
!pip install ludwig

     |████████████████████████████████| 212 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 31.0 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 25.7 MB/s eta 0:00:01
     |████████████████████████████████| 17.7 MB 38.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.5 MB 49.8 MB/s eta 0:00:01
  Created wheel for ludwig: filename=ludwig-0.3-py3-none-any.whl size=283599 sha256=def8b727c6b1b993793877f1d5722132299f432e712088e9ad89d1b0caf2514c
  Stored in directory: /root/.cache/pip/wheels/5f/fd/c6/2ecb6a864020ac1873f9602117a47597eb8c60e44587c3d479
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8915 sha256=c95662ba7c177a42323a833aca92ca4e74c2a427c8a961883a9868cc7be90e2a
  Stored in directory: /root/.cache/pip/wheels/e2/bd/55/048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f3

In [2]:
!HOROVOD_GPU_OPERATIONS=NCCL pip install ludwig[horovod]

     |████████████████████████████████| 3.2 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 465 kB 41.6 MB/s eta 0:00:01
     |████████████████████████████████| 401 kB 43.4 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 55.0 MB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /usr/local/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-nngm0bql/horovod/setup.py'"'"'; __file__='"'"'/tmp/pip-install-nngm0bql/horovod/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-8poxkwpa
       cwd: /tmp/pip-install-nngm0bql/horovod/
  Complete output (1135 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/horovod
  copying horovod/__init__.py -> b

In [3]:
!pip install spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 185 kB 4.9 MB/s eta 0:00:01
  Attempting uninstall: srsly
    Found existing installation: srsly 2.0.1
    Uninstalling srsly-2.0.1:
      Successfully uninstalled srsly-2.0.1
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 12.0 MB 765 kB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-py3-none-any.whl size=12011738 sha256=23af716e5990a509da3f366702519f0ffa4cfe09e38fc55e51c483a2069a27d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-sy_y2vzt/wheels/51/19/da/a3885266a3c241aff0ad2eb674ae058fd34a4870fef1c0a5a0
Successfully built en-core-web-sm
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
import spacy

In [6]:
nlp = spacy.load("en_core_web_sm", disable=['tagger', 'parser', 'ner'])
def tokenizer(text, nlp):

    token_list = []
    doc = nlp(text)
    for token in doc:
        if token.is_stop == False and token.is_punct==False:
            if token.text != ' ':
                token_list.append((token.lemma_).lower())
    str_tokens = ' '.join(token_list)
    return str_tokens

In [7]:
import horovod.tensorflow as hvd

In [8]:
import tensorflow as tf

In [9]:
hvd.init()

In [10]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.visible_device_list = str(hvd.local_rank())

In [11]:
config.gpu_options.visible_device_list

'0'

In [12]:
#!horovodrun --check-build

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [14]:
from ludwig.api import LudwigModel

# Data set

In [60]:
#reading in gdelt master
print('reading in text daya')
df = pd.read_csv('/floyd/home/Capstone/cap_notebooks/data/master_data_set/gdelt_text_tone_complete_oct_22.csv')

display(f'df: {df.shape}')

#display(df.head())

print('setting datatime')
#cleans date time
#df['date_time']  = df['gkgcode'].apply(lambda x: x[:14])



#converts datetime
df['date_time'] = pd.to_datetime(df['DATE'])

#sets datetime index
df.set_index('date_time', inplace=True)

print('filling empty title columns')
#fills missing title with no_title
df['title'].fillna('no_title', inplace = True)

print('tokenizing titles')
#tokenzies title
df['title_tokens'] = df['title'].apply(lambda x: tokenizer(str(x), nlp))

print('joining text and title')
#joins title and text
df['title_text'] = df['title_tokens'] + df['text_tokens']

#display(df.head())
display(f'df: {df.shape}')

#resampling text at 1 hour and combine tokens
print('resampling at 1h')
test_resample = df.resample('1h')['title_text'].agg(lambda column: "".join(column))
test_resample = pd.DataFrame(test_resample)

print(test_resample.head(3))

print('\n')

#getting gdelt scores and calculating mean for every hour
print('resample scores')
scores = scores.resample('1h').mean()

print(scores.head(3))



#combining text data with GDELT tonal scores
test_resample = pd.concat([test_resample, scores], axis=1)

test_resample.reset_index(inplace=True)
      
print(test_resample.head(3))

display(f'clean text shape: {test_resample.shape}')

useful_sample_size = (test_resample['title_text'].str.len()>0).sum()

display(f'text > 0: {useful_sample_size}')

#adding padding to gdelt before joinning with gtrends
print('adding padding to text')
columns = list(test_resample.columns)


#create data padding
date_list = list(pd.date_range(start='2020-10-07 21:00:00', end='2020-10-11 00:00:00', freq='1h'))

data = {'date_time': date_list, 'title_text': 'empty_string'}

df_holder = pd.DataFrame(data,columns=columns)

#appending padding to text data
print('appending new padding to test_resample')
test_resample = test_resample.append(df_holder)

print('reading in google trends')
#read in gtrend data
gtrends = pd.read_csv('/floyd/home/Capstone/cap_notebooks/data/google_trends/gtrends_2015-2020_clean.csv')

display(f'gtrend shape: {gtrends.shape}')

print('grouping gtrends data and removing duplicates')
gtrends = gtrends.groupby('date').mean()

display(f'gtrends duplicates removed: {gtrends.shape}')

print('removing extra google trends data')
gtrends = gtrends['2015-03-02 00:00:00': '2020-10-10 00:00:00']
gtrends.reset_index(inplace=True)
gtrends.drop(['Unnamed: 0', 'isPartial'], axis=1, inplace=True)

print('setting datetime on gtrends')
gtrends['date'] = pd.to_datetime(gtrends['date'])

display(f'gtrends timeframe reduced: {gtrends.shape}')

#merging text and trends
gtrends_gdelt = gtrends.merge(test_resample, how='inner', left_on = 'date', right_on = 'date_time')

#dropping extra data column
gtrends_gdelt.drop('date_time', axis=1, inplace=True)
display(f'gtrends_gdelt shape: {gtrends_gdelt.shape}')
display(f'gtrends_gdelt cols: {gtrends_gdelt.columns}')

#checking complete recores
complete_records = (gtrends_gdelt['title_text'].str.len()>0).sum()
print(f'gtrends_gdelt complete records: {complete_records}')

#binarizing depression
gtrends_gdelt['depression_binary'] = np.where(gtrends_gdelt['depression'] >= 36, 1, 0)
dep_val_count = gtrends_gdelt['depression_binary'].value_counts()

print(f'binary_depression value counts: {dep_val_count}')

#shifting trends data to so target is now 12 hours later that text feature data
print('shifting google trends by -24 houors')
gtrends_gdelt['shifted_12h'] = gtrends_gdelt['depression_binary'].shift(-12)

#dropping NA rows, which should just be 24 end rows
gtrends_gdelt.dropna(inplace=True)

#grapping complete rows

print('setting final df')
gtrends_text_final = gtrends_gdelt[gtrends_gdelt['title_text'].str.len()>12]

print(gtrends_text_final.info())
print('\n')
display(gtrends_text_final.head())
print('\n')

#filtering dataset
x_y = gtrends_text_final[['title_text', 'avg_tone', 'pos_words', 'neg_words', 'polarity', 'act_ref_density', 'self_group_density', 'word_count', 'shifted_12h']]
x_y.head()


reading in text daya


'df: (214706, 12)'

setting datatime
filling empty title columns
tokenizing titles
joining text and title


'df: (214706, 14)'

resampling at 1h
                                                            title_text
date_time                                                             
2015-03-02 10:00:00  america clean energy laggardanswer resound myr...
2015-03-02 11:00:00                                                   
2015-03-02 12:00:00                                                   


resample scores
                     avg_tone  pos_words  neg_words  polarity  \
DATE                                                            
2015-03-02 10:00:00  0.350631   2.734923   2.384292  5.119215   
2015-03-02 11:00:00       NaN        NaN        NaN       NaN   
2015-03-02 12:00:00       NaN        NaN        NaN       NaN   

                     act_ref_density  self_group_density  word_count  
DATE                                                                  
2015-03-02 10:00:00        16.760168            0.420757      1186.0  
2015-03-02 11:00:00              NaN                 NaN         NaN  


...

'gtrends_gdelt shape: (48528, 14)'

"gtrends_gdelt cols: Index(['date', 'depression', 'anxiety', 'government', 'politics', 'democracy',\n       'title_text', 'avg_tone', 'pos_words', 'neg_words', 'polarity',\n       'act_ref_density', 'self_group_density', 'word_count'],\n      dtype='object')"

gtrends_gdelt complete records: 29441
binary_depression value counts: 1    24881
0    23647
Name: depression_binary, dtype: int64
shifting google trends by -24 houors
setting final df
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29389 entries, 0 to 48475
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                29389 non-null  datetime64[ns]
 1   depression          29389 non-null  float64       
 2   anxiety             29389 non-null  float64       
 3   government          29389 non-null  float64       
 4   politics            29389 non-null  float64       
 5   democracy           29389 non-null  float64       
 6   title_text          29389 non-null  object        
 7   avg_tone            29389 non-null  float64       
 8   pos_words           29389 non-null  float64       
 9   neg_words           29389 non-null  float64       
 10  polarity            29389 non-

,date,depression,anxiety,government,politics,democracy,title_text,avg_tone,pos_words,neg_words,polarity,act_ref_density,self_group_density,word_count,depression_binary,shifted_12h
0,2015-03-02 10:00:00,15.0,16.0,71.0,4.0,2.0,america clean energy laggardanswer resound myr...,0.350631,2.734923,2.384292,5.119215,16.760168,0.420757,1186.000000,0,0.0
5,2015-03-02 15:00:00,18.0,13.0,42.0,6.0,5.0,watch meet press treat climate change big joke...,-0.952381,3.492063,4.444444,7.936508,26.984127,1.428571,576.000000,0,0.0
6,2015-03-02 16:00:00,17.0,13.0,41.0,6.0,5.0,no_titlemary bowerman usa today network visito...,0.000000,1.814059,1.814059,3.628118,25.396825,0.000000,405.000000,0,0.0
8,2015-03-02 18:00:00,18.0,14.0,39.0,6.0,4.0,russian energy deal come contentious timemr. f...,-1.147541,1.803279,2.950820,4.754098,19.508197,0.409836,1119.000000,0,0.0
10,2015-03-02 20:00:00,17.0,13.0,35.0,5.0,4.0,climate change cause syrian civil warclimate c...,-5.368873,0.909366,6.278239,7.187605,24.159231,0.123712,616.333333,0,0.0


,title_text,avg_tone,pos_words,neg_words,polarity,act_ref_density,self_group_density,word_count,shifted_12h
0,america clean energy laggardanswer resound myr...,0.350631,2.734923,2.384292,5.119215,16.760168,0.420757,1186.000000,0.0
5,watch meet press treat climate change big joke...,-0.952381,3.492063,4.444444,7.936508,26.984127,1.428571,576.000000,0.0
6,no_titlemary bowerman usa today network visito...,0.000000,1.814059,1.814059,3.628118,25.396825,0.000000,405.000000,0.0
8,russian energy deal come contentious timemr. f...,-1.147541,1.803279,2.950820,4.754098,19.508197,0.409836,1119.000000,0.0
10,climate change cause syrian civil warclimate c...,-5.368873,0.909366,6.278239,7.187605,24.159231,0.123712,616.333333,0.0


# Model 1 - Ludwig with Bert

- Attempted to apply Bert transformer via Ludwig to text data
- Model ran but I aborted because it was taking longer than my patience would allow
- This ludwig model only has one hidden layer, which hadn't done that well on previous iterations, which is why I decided not continue down this path
- I think there is a lot of potential here as well, but I need to figure out how to customize network architecture before devoting computing resources to Bert

In [15]:
from transformers import PreTrainedTokenizer

In [16]:
tokenizer= PreTrainedTokenizer(max_length=512)

In [17]:
model_definition = {
    'input_features': [
        {'name': 'title_text', 'type': 'text', 'level': 'word', 'encoder': 'bert', 'max_len': 512,
         'config_path': '/floyd/input/bert/uncased_L-24_H-1024_A-16/bert_config.json',
         'checkpoint_path': 'input/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001',
         'preprocessing': {'tokenizer': 'bert', 
                           'vocab_file': '/floyd/input/bert/uncased_L-24_H-1024_A-16/vocab.txt',
                          'padding_symbol': '[PAD]',
                          'unknown_symbol': '[UNK]'}},
    ],
    'output_features': [
        {'name': 'shifted_12h', 'type': 'binary'},
    ]
}

In [ ]:
train_stats = model.train(dataset=x_y, use_horovod=True, gpus='0')


# Model 2 - Ludwig with Stanford GLoVe word embedding
- Ludwig does not support the Google word embedding .pb models I was using in other neural nets
- All ludwig models only have one hidden layer (and I have yet to figure out to build a more complex architecture with this package)

In [21]:
model_definition = {
    'input_features': [
        {'name': 'title_text', 'type': 'text', 'level': 'word', 'tied_weights': 'null',
         'encoder': 'embed', 'pretrained_model_name_or_path': '/floyd/input/glove/glove.6B.50d.txt',
         }],
            'output_features': [
        {'name': 'shifted_12h', 'type': 'binary'}
    ]
}

In [22]:
model = LudwigModel(model_definition, use_horovod=True, gpus='0')

In [23]:
train_stats = model.train(dataset=x_y, use_horovod=True, gpus='0')

/usr/local/lib/python3.7/site-packages/ludwig/data/preprocessing.py:1045: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  computed_fill_value,


# Model 3 - Ludwig Stacked Ensemble Neural Net combining Glove Word embedding for text and dense layers for GDELT sentiment

- training acc: 83 percent
- test acc: 53 percent

In [61]:
x_y.columns

Index(['title_text', 'avg_tone', 'pos_words', 'neg_words', 'polarity',
       'act_ref_density', 'self_group_density', 'word_count', 'shifted_12h'],
      dtype='object')

In [65]:
#define model
model_definition = {
    'input_features': [
        {'name': 'title_text', 'type': 'text', 'level': 'word', 'tied_weights': 'null',
         'encoder': 'embed', 'representation': 'dense', 'num_layers': 4, 'weights_regulizer': 'l2', 'dropout': 0.2, 'fc_size': 6
         , 'pretrained_model_name_or_path': '/floyd/input/glove/glove.6B.50d.txt',
         'preprocessing': {'word_tokenizer': 'space_punct', 'word_most_common': 30000}
         },
    {'name':'avg_tone', 'type': 'numerical', 'encoder': 'dense'},
    {'name':'pos_words', 'type': 'numerical', 'encoder': 'dense'},
    {'name':'neg_words', 'type': 'numerical', 'encoder': 'dense'},
    {'name':'polarity', 'type': 'numerical', 'encoder': 'dense'},
    {'name':'act_ref_density', 'type': 'numerical', 'encoder': 'dense'},
    {'name':'self_group_density', 'type': 'numerical', 'encoder': 'dense'},
    {'name':'word_count', 'type': 'numerical', 'encoder': 'dense'}
    ],
    'output_features': [
        {'name': 'shifted_12h', 'type': 'binary'},
    ]

}

In [66]:
#instanstiate model
model = LudwigModel(model_definition, use_horovod=True, gpus='0')

In [67]:
#train model
train_stats = model.train(dataset=x_y, use_horovod=True, gpus='0')

/usr/local/lib/python3.7/site-packages/ludwig/data/preprocessing.py:1045: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  computed_fill_value,
